# TORCH 사용자를 위한 PYTORCH 따라치기

- Author : Soumith Chintala
- 번역 : 박정환
- 재구성 : 박범진
- 원본 : [링크](https://tutorials.pytorch.kr/beginner/former_torchies_tutorial.html)

1. torch Tensor를 사용해보고 (Lua)Torch와의 차이 알아보기
2. autograd 패키지 사용
3. 신경망 구성
  - 합성공 신경망 구성
  - 순환 신경망 구성

## Tensor

In [1]:
import torch
a = torch.empty(5, 7, dtype=torch.float)

In [2]:
a = torch.randn(5, 7, dtype=torch.double)
print(a)
print(a.size())

tensor([[-0.8321, -0.5338, -1.3557, -1.3160,  1.1388,  1.9611, -0.3921],
        [ 1.2003,  0.4142,  0.4677, -0.2806,  0.5839,  0.1989,  1.4824],
        [-1.1681, -0.6514,  0.6338, -0.6966,  1.7018,  0.7653,  0.1988],
        [-0.5449,  2.0696,  0.2691,  1.4259,  0.0579,  0.0491, -1.9611],
        [-1.1346,  0.4757,  0.5323,  0.5826,  2.0657, -1.6675,  0.8987]],
       dtype=torch.float64)
torch.Size([5, 7])


- 평균 0, 분산 1의 정규분포를 따르는 무작위 숫자로 double tensor 초기화

### IN-PLACE / OUT-OF-PLACE

In [3]:
a.fill_(3.5)

tensor([[3.5000, 3.5000, 3.5000, 3.5000, 3.5000, 3.5000, 3.5000],
        [3.5000, 3.5000, 3.5000, 3.5000, 3.5000, 3.5000, 3.5000],
        [3.5000, 3.5000, 3.5000, 3.5000, 3.5000, 3.5000, 3.5000],
        [3.5000, 3.5000, 3.5000, 3.5000, 3.5000, 3.5000, 3.5000],
        [3.5000, 3.5000, 3.5000, 3.5000, 3.5000, 3.5000, 3.5000]],
       dtype=torch.float64)

In [4]:
b = a.add(4.0)

In [5]:
print(a)
print(b)

tensor([[3.5000, 3.5000, 3.5000, 3.5000, 3.5000, 3.5000, 3.5000],
        [3.5000, 3.5000, 3.5000, 3.5000, 3.5000, 3.5000, 3.5000],
        [3.5000, 3.5000, 3.5000, 3.5000, 3.5000, 3.5000, 3.5000],
        [3.5000, 3.5000, 3.5000, 3.5000, 3.5000, 3.5000, 3.5000],
        [3.5000, 3.5000, 3.5000, 3.5000, 3.5000, 3.5000, 3.5000]],
       dtype=torch.float64)
tensor([[7.5000, 7.5000, 7.5000, 7.5000, 7.5000, 7.5000, 7.5000],
        [7.5000, 7.5000, 7.5000, 7.5000, 7.5000, 7.5000, 7.5000],
        [7.5000, 7.5000, 7.5000, 7.5000, 7.5000, 7.5000, 7.5000],
        [7.5000, 7.5000, 7.5000, 7.5000, 7.5000, 7.5000, 7.5000],
        [7.5000, 7.5000, 7.5000, 7.5000, 7.5000, 7.5000, 7.5000]],
       dtype=torch.float64)


### 0 Indexing

In [6]:
b = a[0, 3] # select 1st row, 4th column from a

In [7]:
print(b)

tensor(3.5000, dtype=torch.float64)


In [8]:
b = a[:, 3:5] # selects all rows, 4th column and 5th column from a

In [9]:
print(b)

tensor([[3.5000, 3.5000],
        [3.5000, 3.5000],
        [3.5000, 3.5000],
        [3.5000, 3.5000],
        [3.5000, 3.5000]], dtype=torch.float64)


### 카멜표기법 없음

In [10]:
x = torch.ones(5, 5)
print(x)

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])


In [11]:
z = torch.empty(5, 2)
z[:, 0] = 10
z[:, 1] = 100
print(z)

tensor([[ 10., 100.],
        [ 10., 100.],
        [ 10., 100.],
        [ 10., 100.],
        [ 10., 100.]])


In [12]:
x.index_add_(1, torch.tensor([4, 0], dtype=torch.long), z)
print(x)

tensor([[101.,   1.,   1.,   1.,  11.],
        [101.,   1.,   1.,   1.,  11.],
        [101.,   1.,   1.,   1.,  11.],
        [101.,   1.,   1.,   1.,  11.],
        [101.,   1.,   1.,   1.,  11.]])


- z의 첫 열을 x의 4번째 열과 더하고 z의 두 번째 열을 x의 0번째 열과 더함

## AUTOGRAD

- 자동 미분을 위해 tape 기반 시스템 사용
- 순전파 단계에서 tape는 수행하는 모뎐 연산을 기억하고 역전파 단계에서 연산들을 재생함

### 이력을 추적하는 TENSOR

- requires_grad=True로 설정된 Tensor의 연산은 기록
- 역전파 단계 연산 후에, 이 변수에 대한 변화도는 .grad에 누적
- 미분을 계산하기 위해서는 Tensor의 .backward() 호출

In [13]:
import torch

In [14]:
x = torch.ones(2, 2, requires_grad = True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [15]:
print(x.data)

tensor([[1., 1.],
        [1., 1.]])


In [16]:
print(x.grad_fn)

None


In [17]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward>)


- 사용자가 만든 Tensor(x)는 grad_fn이 None으로 지정
- 연산의 결과로 생성된 y는 grad_fn을 가짐

In [18]:
z = y*y*3
out = z.mean()
print(z)
print(out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward>)
tensor(27., grad_fn=<MeanBackward1>)


In [19]:
a = torch.randn(2, 2)
a = ((a*3) / (a-1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a*a).sum()
print(b.grad_fn)

False
True


### 변화도(GRADIENT)

- d/d

In [20]:
out.backward()
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


- 특정 부분에 대한 역전파 연산을 2번하고 싶다면, 첫 연산 단계에서 retain_variables = True 값을 넘겨줘야 함

In [25]:
x = torch.ones(2, 2, requires_grad=True)
y = x + 2
y.backward(torch.ones(2, 2), retain_graph=True)
# retain_variables flag는 내부 버퍼가 사라지는 것을 막아줌
print(x.grad)

tensor([[1., 1.],
        [1., 1.]])


In [26]:
z = y * y
print(z)

tensor([[9., 9.],
        [9., 9.]], grad_fn=<ThMulBackward>)


In [27]:
gradient = torch.randn(2, 2)
y.backward(gradient)
# retain_variable을 지정하지 않으면 오류 발생
print(x.grad)

tensor([[ 1.4620,  0.5246],
        [ 1.6958, -1.0808]])


### Tensor 추적 방지

In [28]:
print(x.requires_grad)
print((x**2).requires_grad)

with torch.no_grad():
    print((x**2).requires_grad)

True
True
False


## NN 패키지

- 상태는 모듈 내에 저장되지 않고, 신경망 그래프 상에 존재
- 동일한 linear layer를 여러 차례 호출하면 순환신경망이 됨

![torch pytorch 비교](https://tutorials.pytorch.kr/_images/torch-nn-vs-pytorch-nn.png)

- What you see is what you get.

## 예제1 : 합성곱 신경망

In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [33]:
class MNISTConvNet(nn.Module):
    
    def __init__(self):
        super(MNISTConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(10, 20, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        # 어떻게 저장되는지는 아래의 '신경망 사용' 참고
        
    def forward(self, input):
        x = self.pool1(F.relu(self.conv1(input)))
        x = self.pool2(F.relu(self.conv2(x)))
        
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x

### 신경망 사용

In [34]:
net = MNISTConvNet()
print(net)

MNISTConvNet(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)


- torch.nn은 미니 배치만 지원
- nnConv2D는 nSamples x nChannels x Height x Width의 4차원 Tensor를 입력으로 함
- 하나의 샘플만 있을 경우 input.unsqueeze(0)을 사용해서 가짜 차원 추가

In [41]:
input = torch.randn(1, 1, 28, 28)
out = net(input)
print(out.size())

torch.Size([1, 10])


In [42]:
target = torch.tensor([3], dtype = torch.long)
loss_fn = nn.CrossEntropyLoss() # LogSoftmax + ClassNLL Loss
err = loss_fn(out, target)
err.backward()
print(err)

tensor(2.0805, grad_fn=<NllLossBackward>)


In [44]:
print(net.conv1.weight.grad.size())

torch.Size([10, 1, 5, 5])


- 개별 계층의 weight와 gradient에 접근

In [45]:
print(net.conv1.weight.data.norm()) # norm of the weight
print(net.conv1.weight.grad.data.norm()) # norm of the gradients

tensor(1.8393)
tensor(0.7516)


### 순방향/역방향 함수 훅(HOOK)

In [46]:
def printnorm(self, input, output):
    # input is a tuple of packed inputs
    # output is a Tensor. output.data is the Tensor we are interested
    print('Inside ' + self.__class__.__name__ + ' forward')
    print('')
    print('input: ', type(input))
    print('input[0]: ', type(input[0]))
    print('output: ', type(output))
    print('')
    print('input size: ', input[0].size())
    print('output size: ', output.data.size())
    print('output norm: ', output.data.norm())
    
    
net.conv2.register_forward_hook(printnorm)

out = net(input)

Inside Conv2d forward

input:  <class 'tuple'>
input[0]:  <class 'torch.Tensor'>
output:  <class 'torch.Tensor'>

input size:  torch.Size([1, 10, 12, 12])
output size:  torch.Size([1, 20, 8, 8])
output norm:  tensor(13.7558)


In [47]:
def printgradnorm(self, grad_input, grad_output):
    print('Inside ' + self.__class__.__name__ + ' backward')
    print('Inside class: ' + self.__class__.__name__)
    print('')
    print('grad_input: ', type(grad_input))
    print('grad_input[0]: ', type(grad_input[0]))
    print('grad_output: ', type(grad_output))
    print('grad_output[0]: ', type(grad_output[0]))
    print('')
    print('grad_input size: ', grad_input[0].size())
    print('grad_output size: ', grad_output[0].size())
    print('grad_input norm: ', grad_input[0].norm())
    
net.conv2.register_backward_hook(printgradnorm)

out = net(input)
err = loss_fn(out, target)
err.backward

Inside Conv2d forward

input:  <class 'tuple'>
input[0]:  <class 'torch.Tensor'>
output:  <class 'torch.Tensor'>

input size:  torch.Size([1, 10, 12, 12])
output size:  torch.Size([1, 20, 8, 8])
output norm:  tensor(13.7558)


<bound method Tensor.backward of tensor(2.0805, grad_fn=<NllLossBackward>)>

## 예제2 : 순환 신경망(RECURRENT NETS)

In [49]:
class RNN(nn.Module):
    
    # you can also accept arguments in your model constructor
    def __init__(self, data_size, hidden_size, output_size):
        super(RNN, self).__init__()
        
        self.hidden_size = hidden_size
        input_size = data_size + hidden_size
        
        self.i2h = nn.Linear(input_size, hidden_size)
        self.h2o = nn.Linear(hidden_size, output_size)
        
    def forward(self, data, last_hidden):
        input = torch.cat((data, last_hidden), 1)
        hidden = self.i2h(input)
        output = self.h2o(hidden)
        return hidden, output
    
rnn = RNN(50, 20, 10)

In [50]:
rnn

RNN(
  (i2h): Linear(in_features=70, out_features=20, bias=True)
  (h2o): Linear(in_features=20, out_features=10, bias=True)
)

In [53]:
loss_fn = nn.MSELoss()

batch_size = 10
TIMESTEPS = 5

# Create some fake data
batch = torch.randn(batch_size, 50)
hidden = torch.zeros(batch_size, 20)
target = torch.zeros(batch_size, 10)

loss = 0
for t in range(TIMESTEPS):
    hidden, output = rnn(batch, hidden)
    loss += loss_fn(output, target)
loss.backward()